In [17]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [18]:
# Some constants for the model
alpha = 0.1 # Hospitalization rate for non-vaccinated
beta = 0.02 ;# Hospitalization rate for vaccinated 
gamma = 0.1 # State can only vaccinate 10% of population per week
file_name = "solution_real" ; # JLD file to save the solution variables in for later analysis 

In [19]:
data_folder = "real_world_data"
# Load csv files
case_file = data_folder * "/" * "case_data.csv"
population_file = data_folder * "/" * "population_data.csv"
vaccine_file = data_folder * "/" * "vaccine_data.csv"

case_data = CSV.read(case_file, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)


P = Matrix(population_data)
C = Matrix(case_data)
V = Matrix(vaccine_data)
# convert population data to matrix

C =C'
V = V[:,1]
P = P[1,:]

println("C: ", size(C))
println("P: ", size(P))
println("V: ", size(V))

C: (50, 52)
P: (50,)
V: (52,)


In [ ]:
n_states = 1:size(case_data, 2)
n_weeks = 1:size(case_data, 1) 
n_weeks_2 = 1:size(case_data, 1)-1 

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)

In [ ]:
model = Model(Gurobi.Optimizer)

In [ ]:
@variable(model, X[n_states, n_weeks] >= 0);
X

In [ ]:
@variable(model, Y[n_states, n_weeks] >= 0);
Y

In [ ]:
@variable(model, W[n_states, n_weeks] >= 0);
W

In [ ]:
@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks)) ; 

In [ ]:
@constraint(model, hospitalizations[i in n_states,j in n_weeks], Y[i,j] == alpha * C[i,j] * (1 - W[i,j]) + beta * C[i,j] * W[i,j]) 

In [ ]:
@constraint(model, vaccinated[i in n_states,j in n_weeks_2], W[i,j+1] == W[i,j] + X[i,j]/ P[i] ) 

In [ ]:
# Initial values of W are  0 
@constraint(model, initial[i in n_states], W[i,1] == 0) 

In [ ]:
# W should be between 0 and 1
@constraint(model, bounds[i in n_states,j in n_weeks], W[i,j] <= 1)

In [ ]:
@constraint(model,vaccine_supply[j in n_weeks], sum(X[i,j] for i in n_states) <= V[j]) 

In [ ]:
@constraint(model,state_vaccination_capacity[i in n_states, j in n_weeks], X[i,j] <= gamma * P[i])  

In [ ]:
optimize!(model)

In [ ]:
objective_value(model)


In [ ]:
W = value.(W)
X = value.(X)
Y = value.(Y)

X_soln = zeros(total_states, total_weeks)
Y_soln = zeros(total_states, total_weeks)
W_soln = zeros(total_states, total_weeks)
# Compute distances between all pairs of points
for i in n_states
    for j in n_weeks
        X_soln[i,j] = X[i,j]
        Y_soln[i,j] = Y[i,j]
        W_soln[i,j] = W[i,j]
    end
end

# save the solutions as a jld file
save("solution_real.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)

In [ ]:
print(Y)